In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [125]:
class DataProcessing:
    
    def read_data(self,file_path):
    
        self.data = pd.read_csv(file_path,sep='\t')
        return self.data
    
    
    def cleaning_data(self,df):
        
        df = df.dropna()
        return df
    
    def get_data(self):
        
        data = self.read_data('marketing_campaign.csv')
        result = self.cleaning_data(data)
        return result

In [154]:
class CustomerSegmentation:

    @staticmethod
    def gen_feature(df):
        
        
        df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"])
        
        dates = []
        for v in  df["Dt_Customer"]:
            v = v.date()
            dates.append(v)
            
        days = []
        
        d1 = max(dates)
        for i in dates:
            delta = d1 - i
            days.append(delta)

        df["Customer_For"] = days
        df["Customer_For"] = pd.to_numeric(df["Customer_For"], errors="coerce")
        
        df["Age"] = max(dates).year-df["Year_Birth"]
        
        df["spend"] = df[["MntWines",
                                          "MntFruits",
                                          "MntMeatProducts",
                                          "MntFishProducts",
                                          "MntSweetProducts",
                                          "MntGoldProds"]].sum(axis=1)

        df["Living_With"]=df["Marital_Status"].replace(
                                                    {"Married":"Partner",
                                                     "Together":"Partner",
                                                     "Absurd":"Alone",
                                                     "Widow":"Alone",
                                                     "YOLO":"Alone",
                                                     "Divorced":"Alone",
                                                     "Single":"Alone",})
        
        
        df["Children"] = df["Kidhome"]+df["Teenhome"]
        
        df['Family_Size'] = df['Living_With'].replace({'Alone':1,'Partner':2}) + df["Children"]
        
        df['Is_Parent'] = np.where(df.Children > 0 , 1,0)
        
        df["Education"]=df["Education"].replace(
                                            {"Basic":"Undergraduate",
                                             "2n Cycle":"Undergraduate",
                                             "Graduation":"Graduate",
                                             "Master":"Postgraduate",
                                             "PhD":"Postgraduate"})
        
        
        df=df.rename(
                columns={"MntWines": "Wines",
                         "MntFruits":"Fruits",
                         "MntMeatProducts":"Meat",
                         "MntFishProducts":"Fish",
                         "MntSweetProducts":"Sweets",
                         "MntGoldProds":"Gold"})
        
        
        to_drop = ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID"]
        df = df.drop(to_drop, axis=1)
        
        return df
        
        
    def get_fetures(self):
        obj = DataProcessing()
        df = obj.get_data()


        df_result = self.gen_feature(df)
        return df_result

In [155]:
obj = CustomerSegmentation()

In [156]:
obj.get_fetures()

,Education,Income,Kidhome,Teenhome,Recency,Wines,Fruits,Meat,Fish,Sweets,...,AcceptedCmp2,Complain,Response,Customer_For,Age,spend,Living_With,Children,Family_Size,Is_Parent
0,Graduate,58138.0,0,0,58,635,88,546,172,88,...,0,0,1,83894400000000000,57,1617,Alone,0,1,0
1,Graduate,46344.0,1,1,38,11,1,6,2,1,...,0,0,0,10800000000000000,60,27,Alone,2,3,1
2,Graduate,71613.0,0,0,26,426,49,127,111,21,...,0,0,0,40780800000000000,49,776,Partner,0,2,0
3,Graduate,26646.0,1,0,26,11,4,20,10,3,...,0,0,0,5616000000000000,30,53,Partner,1,3,1
4,Postgraduate,58293.0,1,0,94,173,43,118,46,27,...,0,0,0,27734400000000000,33,422,Partner,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,Graduate,61223.0,0,1,46,709,43,182,42,118,...,0,0,0,46742400000000000,47,1341,Partner,1,3,1
2236,Postgraduate,64014.0,2,1,56,406,0,30,0,0,...,0,0,0,5270400000000000,68,444,Partner,3,5,1
2237,Graduate,56981.0,0,0,91,908,48,217,32,12,...,0,0,0,27216000000000000,33,1241,Alone,0,1,0
2238,Postgraduate,69245.0,0,1,8,428,30,214,80,30,...,0,0,0,27302400000000000,58,843,Partner,1,3,1
